<a href="https://colab.research.google.com/github/Zerldas/Python_Exercise/blob/main/VGG/VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.preprocessing import image
from google.colab import files

In [ ]:
# Chuẩn bị dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = resize(x_train, (64, 64))
x_test = resize(x_test, (64, 64))

x_train, x_test = x_train / 255.0, x_test / 255.0 # [0; 1]

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
def create_vgg(base_model):
    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(64,64,3))
vgg16_base.summary()
model_vgg16 = create_vgg(vgg16_base)

In [ ]:
EPOCHS = 10
BATCH = 32

print("\n======= VGG16 =======")
vgg16 = model_vgg16.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH, validation_data=(x_test, y_test), verbose=2)

In [ ]:
os.makedirs("test_images", exist_ok=True)

uploaded = files.upload()

for name, data in uploaded.items():
    with open(f"test_images/{name}", "wb") as f:
        f.write(data)
print(" Đã upload xong ảnh test!")

In [ ]:
class_names = [
  'airplane', 'automobile', 'bird', 'cat', 'deer',
  'dog', 'frog', 'horse', 'ship', 'truck'
]


def predict_image(model, folder_path, target_size=(64, 64)):
    files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    for f in files:
        img_path = os.path.join(folder_path, f)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        pred = model.predict(img_array)
        cls = np.argmax(pred)
        confidence = np.max(pred) * 100

        plt.imshow(tf.keras.preprocessing.image.load_img(img_path))
        plt.title(f"{f} → {class_names[cls]} ({confidence:.1f}%)")
        plt.axis("off")
        plt.show()

In [ ]:
# Test VGG16
predict_image(model_vgg16, "test_images/")